In [1]:
! pip install git+git://github.com/frehseg/nvcc4jupyter.git

  Cloning git://github.com/frehseg/nvcc4jupyter.git to /tmp/pip-req-build-uw3pn5hk
  Running command git clone -q git://github.com/frehseg/nvcc4jupyter.git /tmp/pip-req-build-uw3pn5hk
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.1-cp36-none-any.whl size=2095 sha256=cd9ce27ebcf6138adc9de45e51b2373c40f6e7ec717ac999e5956ca6a2dd40d8
  Stored in directory: /tmp/pip-ephem-wheel-cache-aap2jdfv/wheels/a4/a5/24/17a2b61f9a725a10155cc6fca753aae28436921df21fa16114
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin

## Basic matrix multiplication (non tiled algorithm)

### TILE_WIDTH = 32

In [37]:
%%cu
#include <stdlib.h>
#include <stdio.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>
#include <cublas_v2.h>

#define TILE_WIDTH 32
#define SIZE 20
/********************** kernel **************************/
__global__
void matmul_basique(float *A, float *B, float *C, int nb_ColA, int nb_ColB, int nb_LigneA, int nb_LigneB)
{
  int row = blockIdx.y*blockDim.y+threadIdx.y;
  int col = blockIdx.x*blockDim.x+threadIdx.x;
  float product_val = 0;
  if(row < nb_ColA && col < nb_LigneA){
    for(int k=0; k<nb_ColA;k++) {
      product_val += A[row*nb_ColA+k]*B[k*nb_ColA+col];
    }
    C[row*nb_ColA+col] = product_val;
  }
}

/********************** Host functions **************************/

/*For debugging purpose*/
void affiche_tab(char *chaine, float *tab, int len){
   int k;
   printf("\nLes 10 premiers de %s: \n",chaine);
   for (k=0; k<10; k++) 
      printf("%.2f ",tab[k]);
   printf("\nLes 10 derniers: \n");
   for (k=0; k<len; k++){
       if(tab[k] == 0){
           printf("%i ", k);
           break;
       }
   } 
   printf("\n");
}

/********************** main **************************/
int main(void)
{
  float *A, *B, *C, *gpu_A, *gpu_B, *gpu_C;
  int nbLigneA, nbLigneB, nbColA, nbColB;
  float milliseconds = 0.0;
  cudaEvent_t start, stop ;
  int nIter = 100;
  
  nbLigneA = TILE_WIDTH * SIZE;
  nbLigneB = TILE_WIDTH * SIZE;
  nbColA = TILE_WIDTH * SIZE;
  nbColB = TILE_WIDTH * SIZE;

  dim3 DimGrid(SIZE, SIZE, 1);
  dim3 DimBlock(TILE_WIDTH, TILE_WIDTH, 1);
 
  A = (float*) malloc(nbLigneA * nbColA * sizeof(float));
  B = (float*) malloc(nbLigneB * nbColB * sizeof(float));
  C = (float*) malloc(nbLigneA * nbColB * sizeof(float));

  /*Allocation de l'espace pour le GPU */
  cudaMalloc((void**) &gpu_A, nbLigneA * nbColA * sizeof(float));
  cudaMalloc((void**) &gpu_B, nbLigneB * nbColB * sizeof(float));
  cudaMalloc((void**) &gpu_C, nbLigneA * nbColB * sizeof(float));  
 
  /* Initialisation de A et B*/
  for (int i = 0; i < nbLigneA * nbColA; i++) {
    A[i] = 1.0;
  }

  for (int i = 0; i < nbLigneB * nbColB; i++) {
    B[i] = 2.0;
  }

 /* Copie de gpu_A et gpu_B sur le GPU */
  cudaMemcpy(gpu_A, A, nbLigneA * nbColA * sizeof(float), cudaMemcpyHostToDevice) ;
  cudaMemcpy(gpu_B, B, nbLigneB * nbColB * sizeof(float), cudaMemcpyHostToDevice) ;
  
/* Moyenne du temps d'exécution de la multiplication de matrice basique */
  for(int i=0; i<nIter; i++){
    float tmp_timer = 0.0;
    /* Lancement du kernel avec mesure du temps */
    cudaEventCreate(&start) ; cudaEventCreate(&stop) ;
    cudaEventRecord(start) ;
  
    matmul_basique<<<DimGrid, DimBlock>>>(gpu_A, gpu_B, gpu_C, nbLigneA, nbLigneB, nbColA, nbColB);
  
    cudaEventRecord(stop) ;
    cudaEventSynchronize(stop) ; //Garantit que l’événement s’est exécuté 
    cudaEventElapsedTime(&tmp_timer, start, stop) ;
    milliseconds += tmp_timer;
  }
 
  printf("Average time over %i iterations for basic matrix mutliplication with TILE_WIDTH = %i : %f ms", nIter, TILE_WIDTH, milliseconds/nIter);
  
  /* Copie de gpu_C du GPU sur le CPU */
  cudaMemcpy(C, gpu_C, nbLigneA * nbColB * sizeof(float), cudaMemcpyDeviceToHost) ;
 
  /* Vérification du résultat*/
  float maxError = 0.0f;
  for (int i = 0; i < nbLigneA * nbColB; i++){
      maxError = max(maxError, abs(C[i]- 2*nbLigneB));
  }
  printf("\nMax error: %f\n", maxError);
 
  if(maxError == 0)
    printf("No errors in the computations of the multiplication !");

  /* Libération de la mémoire sur le GPU*/ 
  cudaFree(gpu_A);
  cudaFree(gpu_B);
  cudaFree(gpu_C);

  /* Libération de la mémoire sur le CPU*/
  free(A);
  free(B);
  free(C);
}


Average time over 100 iterations for basic matrix mutliplication with TILE_WIDTH = 32 : 1.547700 ms
Max error: 0.000000
No errors in the computations of the multiplication !


### TILE_WIDTH = 16

In [38]:
%%cu
#include <stdlib.h>
#include <stdio.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>
#include <cublas_v2.h>

#define TILE_WIDTH 16
#define SIZE 20
/********************** kernel **************************/
__global__
void matmul_basique(float *A, float *B, float *C, int nb_ColA, int nb_ColB, int nb_LigneA, int nb_LigneB)
{
  int row = blockIdx.y*blockDim.y+threadIdx.y;
  int col = blockIdx.x*blockDim.x+threadIdx.x;
  float product_val = 0;
  if(row < nb_ColA && col < nb_LigneA){
    for(int k=0; k<nb_ColA;k++) {
      product_val += A[row*nb_ColA+k]*B[k*nb_ColA+col];
    }
    C[row*nb_ColA+col] = product_val;
  }
}

/********************** Host functions **************************/

/*For debugging purpose*/
void affiche_tab(char *chaine, float *tab, int len){
   int k;
   printf("\nLes 10 premiers de %s: \n",chaine);
   for (k=0; k<10; k++) 
      printf("%.2f ",tab[k]);
   printf("\nLes 10 derniers: \n");
   for (k=0; k<len; k++){
       if(tab[k] == 0){
           printf("%i ", k);
           break;
       }
   } 
   printf("\n");
}

/********************** main **************************/
int main(void)
{
  float *A, *B, *C, *gpu_A, *gpu_B, *gpu_C;
  int nbLigneA, nbLigneB, nbColA, nbColB;
  float milliseconds = 0.0;
  cudaEvent_t start, stop ;
  int nIter = 100;
  
  nbLigneA = TILE_WIDTH * SIZE;
  nbLigneB = TILE_WIDTH * SIZE;
  nbColA = TILE_WIDTH * SIZE;
  nbColB = TILE_WIDTH * SIZE;

  dim3 DimGrid(SIZE, SIZE, 1);
  dim3 DimBlock(TILE_WIDTH, TILE_WIDTH, 1);
 
  A = (float*) malloc(nbLigneA * nbColA * sizeof(float));
  B = (float*) malloc(nbLigneB * nbColB * sizeof(float));
  C = (float*) malloc(nbLigneA * nbColB * sizeof(float));

  /*Allocation de l'espace pour le GPU */
  cudaMalloc((void**) &gpu_A, nbLigneA * nbColA * sizeof(float));
  cudaMalloc((void**) &gpu_B, nbLigneB * nbColB * sizeof(float));
  cudaMalloc((void**) &gpu_C, nbLigneA * nbColB * sizeof(float));  
 
  /* Initialisation de A et B*/
  for (int i = 0; i < nbLigneA * nbColA; i++) {
    A[i] = 1.0;
  }

  for (int i = 0; i < nbLigneB * nbColB; i++) {
    B[i] = 2.0;
  }

 /* Copie de gpu_A et gpu_B sur le GPU */
  cudaMemcpy(gpu_A, A, nbLigneA * nbColA * sizeof(float), cudaMemcpyHostToDevice) ;
  cudaMemcpy(gpu_B, B, nbLigneB * nbColB * sizeof(float), cudaMemcpyHostToDevice) ;
  
/* Moyenne du temps d'exécution de la multiplication de matrice basique */
  for(int i=0; i<nIter; i++){
    float tmp_timer = 0.0;
    /* Lancement du kernel avec mesure du temps */
    cudaEventCreate(&start) ; cudaEventCreate(&stop) ;
    cudaEventRecord(start) ;
  
    matmul_basique<<<DimGrid, DimBlock>>>(gpu_A, gpu_B, gpu_C, nbLigneA, nbLigneB, nbColA, nbColB);
  
    cudaEventRecord(stop) ;
    cudaEventSynchronize(stop) ; //Garantit que l’événement s’est exécuté 
    cudaEventElapsedTime(&tmp_timer, start, stop) ;
    milliseconds += tmp_timer;
  }
 
  printf("Average time over %i iterations for basic matrix mutliplication with TILE_WIDTH = %i : %f ms", nIter, TILE_WIDTH, milliseconds/nIter);
  
  /* Copie de gpu_C du GPU sur le CPU */
  cudaMemcpy(C, gpu_C, nbLigneA * nbColB * sizeof(float), cudaMemcpyDeviceToHost) ;
 
  /* Vérification du résultat*/
  float maxError = 0.0f;
  for (int i = 0; i < nbLigneA * nbColB; i++){
      maxError = max(maxError, abs(C[i]- 2*nbLigneB));
  }
  printf("\nMax error: %f\n", maxError);
 
  if(maxError == 0)
    printf("No errors in the computations of the multiplication !");

  /* Libération de la mémoire sur le GPU*/ 
  cudaFree(gpu_A);
  cudaFree(gpu_B);
  cudaFree(gpu_C);

  /* Libération de la mémoire sur le CPU*/
  free(A);
  free(B);
  free(C);
}

Average time over 100 iterations for basic matrix mutliplication with TILE_WIDTH = 16 : 0.287865 ms
Max error: 0.000000
No errors in the computations of the multiplication !


## Tiled Matrix multiplication

### TILE_WIDTH = 32

In [39]:
%%cu
#include <stdlib.h>
#include <stdio.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>
#include <cublas_v2.h>

#define TILE_WIDTH 32
#define SIZE 20
/********************** kernel **************************/
__global__
void matmul_tuile(float *A, float *B, float *C, int nb_ColA, int nb_ColB, int nb_LigneA, int nb_LigneB)
{
  int row = blockIdx.y*blockDim.y+threadIdx.y;
  int col = blockIdx.x*blockDim.x+threadIdx.x;
  //Memoire partagée pour le tuilage
  __shared__ float A_shared[TILE_WIDTH][TILE_WIDTH];
  __shared__ float B_shared[TILE_WIDTH][TILE_WIDTH];
  //Resultats temporaires
  float product_val = 0;
 
  //Rempli la mémoire partagée
  for(int k=0; k < (TILE_WIDTH + nb_ColA - 1)/TILE_WIDTH; k++) {
    //A : tuilage ligne
    if (k*TILE_WIDTH + threadIdx.x < nb_ColA && row < nb_LigneA)
      A_shared[threadIdx.y][threadIdx.x] = A[row*nb_ColA + k*TILE_WIDTH + threadIdx.x];
    else
      A_shared[threadIdx.y][threadIdx.x] = 0.0;

    //B : tuilage colonne
    if (k*TILE_WIDTH + threadIdx.y < nb_LigneB && col < nb_ColB)
      B_shared[threadIdx.y][threadIdx.x] = B[(k*TILE_WIDTH + threadIdx.y)*nb_ColB + col];
    else
      B_shared[threadIdx.y][threadIdx.x] = 0.0;

    //Synchronisation des threads dans chaque bloc
    __syncthreads();

    for (int n = 0; n < TILE_WIDTH; ++n)
      product_val += A_shared[threadIdx.y][n] * B_shared[n][threadIdx.x];

    //Synchronisation des threads dans chaque bloc
     __syncthreads();
    }
      
  //Ecriture du résultat dans C
  if(row < nb_ColA && col < nb_LigneA){
    C[((blockIdx.y * blockDim.y + threadIdx.y)*nb_ColA) + (blockIdx.x * blockDim.x)+ threadIdx.x] = product_val;
  }

}

/********************** Host functions **************************/

/*For debugging purpose*/
void affiche_tab(char *chaine, float *tab, int len){
   int k;
   printf("\nLes 10 premiers de %s: \n",chaine);
   for (k=0; k<10; k++) 
      printf("%.2f ",tab[k]);
   printf("\nLes 10 derniers: \n");
   for (k=0; k<len; k++){
       if(tab[k] == 0){
           printf("%i ", k);
           break;
       }
   } 
   printf("\n");
}

/********************** main **************************/
int main(void)
{
  float *A, *B, *C, *gpu_A, *gpu_B, *gpu_C;
  int nbLigneA, nbLigneB, nbColA, nbColB;
  float milliseconds = 0.0;
  cudaEvent_t start, stop ;
  int nIter = 100;
  
  nbLigneA = TILE_WIDTH * SIZE;
  nbLigneB = TILE_WIDTH * SIZE;
  nbColA = TILE_WIDTH * SIZE;
  nbColB = TILE_WIDTH * SIZE;

  dim3 DimGrid(SIZE, SIZE, 1);
  dim3 DimBlock(TILE_WIDTH, TILE_WIDTH, 1);
 
  A = (float*) malloc(nbLigneA * nbColA * sizeof(float));
  B = (float*) malloc(nbLigneB * nbColB * sizeof(float));
  C = (float*) malloc(nbLigneA * nbColB * sizeof(float));

  /*Allocation de l'espace pour le GPU */
  cudaMalloc((void**) &gpu_A, nbLigneA * nbColA * sizeof(float));
  cudaMalloc((void**) &gpu_B, nbLigneB * nbColB * sizeof(float));
  cudaMalloc((void**) &gpu_C, nbLigneA * nbColB * sizeof(float));  
 
  /* Initialisation de A et B*/
  for (int i = 0; i < nbLigneA * nbColA; i++) {
    A[i] = 1.0;
  }

  for (int i = 0; i < nbLigneB * nbColB; i++) {
    B[i] = 2.0;
  }

 /* Copie de gpu_A et gpu_B sur le GPU */
  cudaMemcpy(gpu_A, A, nbLigneA * nbColA * sizeof(float), cudaMemcpyHostToDevice) ;
  cudaMemcpy(gpu_B, B, nbLigneB * nbColB * sizeof(float), cudaMemcpyHostToDevice) ;
  
/* Moyenne du temps d'exécution de la multiplication de matrice avec algo tuilé */
  for(int i=0; i<nIter; i++){
    float tmp_timer = 0.0;
    /* Lancement du kernel avec mesure du temps */
    cudaEventCreate(&start) ; cudaEventCreate(&stop) ;
    cudaEventRecord(start) ;
  
    matmul_tuile<<<DimGrid, DimBlock>>>(gpu_A, gpu_B, gpu_C, nbLigneA, nbLigneB, nbColA, nbColB);
  
    cudaEventRecord(stop) ;
    cudaEventSynchronize(stop) ; //Garantit que l’événement s’est exécuté 
    cudaEventElapsedTime(&tmp_timer, start, stop) ;
    milliseconds += tmp_timer;
  }
 
  printf("Average time over %i iterations for tiled matrix mutliplication with TILE_WIDTH = %i : %f ms", nIter, TILE_WIDTH, milliseconds/nIter);
  
  /* Copie de gpu_C du GPU sur le CPU */
  cudaMemcpy(C, gpu_C, nbLigneA * nbColB * sizeof(float), cudaMemcpyDeviceToHost) ;
 
  /* Vérification du résultat*/
  float maxError = 0.0f;
  for (int i = 0; i < nbLigneA * nbColB; i++){
      maxError = max(maxError, abs(C[i]- 2*nbLigneB));
  }
  printf("\nMax error: %f\n", maxError);
 
  if(maxError == 0)
    printf("No errors in the computations of the multiplication !");

  /* Libération de la mémoire sur le GPU*/ 
  cudaFree(gpu_A);
  cudaFree(gpu_B);
  cudaFree(gpu_C);

  /* Libération de la mémoire sur le CPU*/
  free(A);
  free(B);
  free(C);
}


Average time over 100 iterations for tiled matrix mutliplication with TILE_WIDTH = 32 : 1.298433 ms
Max error: 0.000000
No errors in the computations of the multiplication !


### TILE_WIDTH = 16

In [40]:
%%cu
#include <stdlib.h>
#include <stdio.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>
#include <cublas_v2.h>

#define TILE_WIDTH 16
#define SIZE 20
/********************** kernel **************************/
__global__
void matmul_tuile(float *A, float *B, float *C, int nb_ColA, int nb_ColB, int nb_LigneA, int nb_LigneB)
{
  int row = blockIdx.y*blockDim.y+threadIdx.y;
  int col = blockIdx.x*blockDim.x+threadIdx.x;
  //Memoire partagée pour le tuilage
  __shared__ float A_shared[TILE_WIDTH][TILE_WIDTH];
  __shared__ float B_shared[TILE_WIDTH][TILE_WIDTH];
  //Resultats temporaires
  float product_val = 0;
 
  //Rempli la mémoire partagée
  for(int k=0; k < (TILE_WIDTH + nb_ColA - 1)/TILE_WIDTH; k++) {
    //A : tuilage ligne
    if (k*TILE_WIDTH + threadIdx.x < nb_ColA && row < nb_LigneA)
      A_shared[threadIdx.y][threadIdx.x] = A[row*nb_ColA + k*TILE_WIDTH + threadIdx.x];
    else
      A_shared[threadIdx.y][threadIdx.x] = 0.0;

    //B : tuilage colonne
    if (k*TILE_WIDTH + threadIdx.y < nb_LigneB && col < nb_ColB)
      B_shared[threadIdx.y][threadIdx.x] = B[(k*TILE_WIDTH + threadIdx.y)*nb_ColB + col];
    else
      B_shared[threadIdx.y][threadIdx.x] = 0.0;

    //Synchronisation des threads dans chaque bloc
    __syncthreads();

    for (int n = 0; n < TILE_WIDTH; ++n)
      product_val += A_shared[threadIdx.y][n] * B_shared[n][threadIdx.x];

    //Synchronisation des threads dans chaque bloc
     __syncthreads();
    }
      
  //Ecriture du résultat dans C
  if(row < nb_ColA && col < nb_LigneA){
    C[((blockIdx.y * blockDim.y + threadIdx.y)*nb_ColA) + (blockIdx.x * blockDim.x)+ threadIdx.x] = product_val;
  }

}

/********************** Host functions **************************/

/*For debugging purpose*/
void affiche_tab(char *chaine, float *tab, int len){
   int k;
   printf("\nLes 10 premiers de %s: \n",chaine);
   for (k=0; k<10; k++) 
      printf("%.2f ",tab[k]);
   printf("\nLes 10 derniers: \n");
   for (k=0; k<len; k++){
       if(tab[k] == 0){
           printf("%i ", k);
           break;
       }
   } 
   printf("\n");
}

/********************** main **************************/
int main(void)
{
  float *A, *B, *C, *gpu_A, *gpu_B, *gpu_C;
  int nbLigneA, nbLigneB, nbColA, nbColB;
  float milliseconds = 0.0;
  cudaEvent_t start, stop ;
  int nIter = 100;
  
  nbLigneA = TILE_WIDTH * SIZE;
  nbLigneB = TILE_WIDTH * SIZE;
  nbColA = TILE_WIDTH * SIZE;
  nbColB = TILE_WIDTH * SIZE;

  dim3 DimGrid(SIZE, SIZE, 1);
  dim3 DimBlock(TILE_WIDTH, TILE_WIDTH, 1);
 
  A = (float*) malloc(nbLigneA * nbColA * sizeof(float));
  B = (float*) malloc(nbLigneB * nbColB * sizeof(float));
  C = (float*) malloc(nbLigneA * nbColB * sizeof(float));

  /*Allocation de l'espace pour le GPU */
  cudaMalloc((void**) &gpu_A, nbLigneA * nbColA * sizeof(float));
  cudaMalloc((void**) &gpu_B, nbLigneB * nbColB * sizeof(float));
  cudaMalloc((void**) &gpu_C, nbLigneA * nbColB * sizeof(float));  
 
  /* Initialisation de A et B*/
  for (int i = 0; i < nbLigneA * nbColA; i++) {
    A[i] = 1.0;
  }

  for (int i = 0; i < nbLigneB * nbColB; i++) {
    B[i] = 2.0;
  }

 /* Copie de gpu_A et gpu_B sur le GPU */
  cudaMemcpy(gpu_A, A, nbLigneA * nbColA * sizeof(float), cudaMemcpyHostToDevice) ;
  cudaMemcpy(gpu_B, B, nbLigneB * nbColB * sizeof(float), cudaMemcpyHostToDevice) ;
  
/* Moyenne du temps d'exécution de la multiplication de matrice avec algo tuilé */
  for(int i=0; i<nIter; i++){
    float tmp_timer = 0.0;
    /* Lancement du kernel avec mesure du temps */
    cudaEventCreate(&start) ; cudaEventCreate(&stop) ;
    cudaEventRecord(start) ;
  
    matmul_tuile<<<DimGrid, DimBlock>>>(gpu_A, gpu_B, gpu_C, nbLigneA, nbLigneB, nbColA, nbColB);
  
    cudaEventRecord(stop) ;
    cudaEventSynchronize(stop) ; //Garantit que l’événement s’est exécuté 
    cudaEventElapsedTime(&tmp_timer, start, stop) ;
    milliseconds += tmp_timer;
  }
 
  printf("Average time over %i iterations for tiled matrix mutliplication with TILE_WIDTH = %i : %f ms", nIter, TILE_WIDTH, milliseconds/nIter);
  
  /* Copie de gpu_C du GPU sur le CPU */
  cudaMemcpy(C, gpu_C, nbLigneA * nbColB * sizeof(float), cudaMemcpyDeviceToHost) ;
 
  /* Vérification du résultat*/
  float maxError = 0.0f;
  for (int i = 0; i < nbLigneA * nbColB; i++){
      maxError = max(maxError, abs(C[i]- 2*nbLigneB));
  }
  printf("\nMax error: %f\n", maxError);
 
  if(maxError == 0)
    printf("No errors in the computations of the multiplication !");

  /* Libération de la mémoire sur le GPU*/ 
  cudaFree(gpu_A);
  cudaFree(gpu_B);
  cudaFree(gpu_C);

  /* Libération de la mémoire sur le CPU*/
  free(A);
  free(B);
  free(C);
}

Average time over 100 iterations for tiled matrix mutliplication with TILE_WIDTH = 16 : 0.198777 ms
Max error: 0.000000
No errors in the computations of the multiplication !


## Matrix multiplication with built-in from CUDA

### TILE_WIDTH = 32

In [41]:
%%cu
#include <stdlib.h>
#include <stdio.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>
#include <cublas_v2.h>

#define TILE_WIDTH 32
#define SIZE 20
/********************** kernel **************************/

/********************** Host functions **************************/

/*For debugging purpose */
void affiche_tab(char *chaine, float *tab, int len){
   int k;
   printf("\nLes 10 premiers de %s: \n",chaine);
   for (k=0; k<10; k++) 
      printf("%.2f ",tab[k]);
   printf("\nLes 10 derniers: \n");
   for (k=0; k<len; k++){
       if(tab[k] == 0){
           printf("%i ", k);
           break;
       }
   } 
   printf("\n");
}

// Multiply the arrays A and B on GPU and save the result in C
// C(m,n) = A(m,k) * B(k,n)
void gpu_blas_mmul(cublasHandle_t &handle, const float *A, const float *B, float *C, const int m, const int k, const int n) {
  int lda=m,ldb=k,ldc=m;
  const float alf = 1;
  const float bet = 0;
  const float *alpha = &alf;
  const float *beta = &bet;
 
  // Do the actual multiplication
  cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, m, n, k, alpha, A, lda, B, ldb, beta, C, ldc);

}

/********************** main **************************/
int main(void)
{
  float *A, *B, *C, *gpu_A, *gpu_B, *gpu_C;
  int nbLigneA, nbLigneB, nbColA, nbColB;
  float milliseconds = 0.0;
  cudaEvent_t start, stop ;
  int nIter = 100;
  
  nbLigneA = TILE_WIDTH * SIZE;
  nbLigneB = TILE_WIDTH * SIZE;
  nbColA = TILE_WIDTH * SIZE;
  nbColB = TILE_WIDTH * SIZE;
 
  A = (float*) malloc(nbLigneA * nbColA * sizeof(float));
  B = (float*) malloc(nbLigneB * nbColB * sizeof(float));
  C = (float*) malloc(nbLigneA * nbColB * sizeof(float));

  /*Allocation de l'espace pour le GPU */
  cudaMalloc((void**) &gpu_A, nbLigneA * nbColA * sizeof(float));
  cudaMalloc((void**) &gpu_B, nbLigneB * nbColB * sizeof(float));
  cudaMalloc((void**) &gpu_C, nbLigneA * nbColB * sizeof(float));  
 
  /* Initialisation de A et B*/
  for (int i = 0; i < nbLigneA * nbColA; i++) {
    A[i] = 1.0;
  }

  for (int i = 0; i < nbLigneB * nbColB; i++) {
    B[i] = 2.0;
  }

 /* Copie de gpu_A et gpu_B sur le GPU */
  cudaMemcpy(gpu_A, A, nbLigneA * nbColA * sizeof(float), cudaMemcpyHostToDevice) ;
  cudaMemcpy(gpu_B, B, nbLigneB * nbColB * sizeof(float), cudaMemcpyHostToDevice) ;
  
  /* Moyenne du temps d'exécution de la multiplication de matrice avec cublasSgemm*/
  for(int i=0; i<nIter; i++){
    float tmp_timer = 0.0;
    // Create a handle for CUBLAS
    cublasHandle_t handle;
    cublasCreate(&handle);

    /* Lancement du kernel avec mesure du temps */
    cudaEventCreate(&start) ; cudaEventCreate(&stop) ;
    cudaEventRecord(start) ;
  
    gpu_blas_mmul(handle, gpu_A, gpu_B, gpu_C, nbLigneA, nbColA, nbColB);

    cudaEventRecord(stop) ;
    cudaEventSynchronize(stop) ; //Garantit que l’événement s’est exécuté 
    cudaEventElapsedTime(&tmp_timer, start, stop) ;
    milliseconds += tmp_timer;
    // Destroy the handle
    cublasDestroy(handle);

  }
 
  printf("Average time over %i iterations for CUDA matrix mutliplication with TILE_WIDTH = %i : %f ms", nIter, TILE_WIDTH, milliseconds/nIter);
  /* Copie de gpu_C du GPU sur le CPU */
  cudaMemcpy(C, gpu_C, nbLigneA * nbColB * sizeof(float), cudaMemcpyDeviceToHost) ;
 
  /* Vérification du résultat*/
  float maxError = 0.0f;
  for (int i = 0; i < nbLigneA * nbColB; i++){
      maxError = max(maxError, abs(C[i]- 2*nbLigneB));
  }
  printf("\nMax error: %f\n", maxError);
 
  if(maxError == 0)
    printf("No errors in the computations of the multiplication !");

  /* Libération de la mémoire sur le GPU*/ 
  cudaFree(gpu_A);
  cudaFree(gpu_B);
  cudaFree(gpu_C);

  /* Libération de la mémoire sur le CPU*/
  free(A);
  free(B);
  free(C);
}

Average time over 100 iterations for CUDA matrix mutliplication with TILE_WIDTH = 32 : 0.261049 ms
Max error: 0.000000
No errors in the computations of the multiplication !


### TILE_WIDTH = 16


In [42]:
%%cu
#include <stdlib.h>
#include <stdio.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>
#include <cublas_v2.h>

#define TILE_WIDTH 16
#define SIZE 20
/********************** kernel **************************/

/********************** Host functions **************************/

void affiche_tab(char *chaine, float *tab, int len){
   int k;
   printf("\nLes 10 premiers de %s: \n",chaine);
   for (k=0; k<10; k++) 
      printf("%.2f ",tab[k]);
   printf("\nLes 10 derniers: \n");
   for (k=0; k<len; k++){
       if(tab[k] == 0){
           printf("%i ", k);
           break;
       }
   } 
   printf("\n");
}

// Multiply the arrays A and B on GPU and save the result in C
// C(m,n) = A(m,k) * B(k,n)
void gpu_blas_mmul(cublasHandle_t &handle, const float *A, const float *B, float *C, const int m, const int k, const int n) {
  int lda=m,ldb=k,ldc=m;
  const float alf = 1;
  const float bet = 0;
  const float *alpha = &alf;
  const float *beta = &bet;
 
  // Do the actual multiplication
  cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, m, n, k, alpha, A, lda, B, ldb, beta, C, ldc);

}

/********************** main **************************/
int main(void)
{
  float *A, *B, *C, *gpu_A, *gpu_B, *gpu_C;
  int nbLigneA, nbLigneB, nbColA, nbColB;
  float milliseconds = 0.0;
  cudaEvent_t start, stop ;
  int nIter = 100;
  
  nbLigneA = TILE_WIDTH * SIZE;
  nbLigneB = TILE_WIDTH * SIZE;
  nbColA = TILE_WIDTH * SIZE;
  nbColB = TILE_WIDTH * SIZE;
 
  A = (float*) malloc(nbLigneA * nbColA * sizeof(float));
  B = (float*) malloc(nbLigneB * nbColB * sizeof(float));
  C = (float*) malloc(nbLigneA * nbColB * sizeof(float));

  /*Allocation de l'espace pour le GPU */
  cudaMalloc((void**) &gpu_A, nbLigneA * nbColA * sizeof(float));
  cudaMalloc((void**) &gpu_B, nbLigneB * nbColB * sizeof(float));
  cudaMalloc((void**) &gpu_C, nbLigneA * nbColB * sizeof(float));  
 
  /* Initialisation de A et B*/
  for (int i = 0; i < nbLigneA * nbColA; i++) {
    A[i] = 1.0;
  }

  for (int i = 0; i < nbLigneB * nbColB; i++) {
    B[i] = 2.0;
  }

 /* Copie de gpu_A et gpu_B sur le GPU */
  cudaMemcpy(gpu_A, A, nbLigneA * nbColA * sizeof(float), cudaMemcpyHostToDevice) ;
  cudaMemcpy(gpu_B, B, nbLigneB * nbColB * sizeof(float), cudaMemcpyHostToDevice) ;
  
  /* Moyenne du temps d'exécution de la multiplication de matrice avec cublasSgemm */
  for(int i=0; i<nIter; i++){
    float tmp_timer = 0.0;
    // Create a handle for CUBLAS
    cublasHandle_t handle;
    cublasCreate(&handle);

    /* Lancement du kernel avec mesure du temps */
    cudaEventCreate(&start) ; cudaEventCreate(&stop) ;
    cudaEventRecord(start) ;
  
    gpu_blas_mmul(handle, gpu_A, gpu_B, gpu_C, nbLigneA, nbColA, nbColB);

    cudaEventRecord(stop) ;
    cudaEventSynchronize(stop) ; //Garantit que l’événement s’est exécuté 
    cudaEventElapsedTime(&tmp_timer, start, stop) ;
    milliseconds += tmp_timer;
    // Destroy the handle
    cublasDestroy(handle);

  }
 
  printf("Average time over %i iterations for CUDA matrix mutliplication with TILE_WIDTH = %i : %f ms", nIter, TILE_WIDTH, milliseconds/nIter);
  /* Copie de gpu_C du GPU sur le CPU */
  cudaMemcpy(C, gpu_C, nbLigneA * nbColB * sizeof(float), cudaMemcpyDeviceToHost) ;
 
  /* Vérification du résultat*/
  float maxError = 0.0f;
  for (int i = 0; i < nbLigneA * nbColB; i++){
      maxError = max(maxError, abs(C[i]- 2*nbLigneB));
  }
  printf("\nMax error: %f\n", maxError);
 
  if(maxError == 0)
    printf("No errors in the computations of the multiplication !");

  /* Libération de la mémoire sur le GPU*/ 
  cudaFree(gpu_A);
  cudaFree(gpu_B);
  cudaFree(gpu_C);

  /* Libération de la mémoire sur le CPU*/
  free(A);
  free(B);
  free(C);
}

Average time over 100 iterations for CUDA matrix mutliplication with TILE_WIDTH = 16 : 0.065900 ms
Max error: 0.000000
No errors in the computations of the multiplication !


## Comparison of results and analysis

Here is the summary of the average computation time over 100 iterations for the 3 different methods of matrix multiplication:

 Time (ms)  | Basic MM | Tiled MM | cublasSgemm MM
--- | --- | --- | ---
TILE_WIDTH = 16 | 0.288 | 0.199 | 0.066
TILE_WIDTH = 32 | 1.548 | 1.299 | 0.261

* As expected, the basic matrix multiplaction is slower than the tiled version of matrix multiplation. As seen in the slides, this is due to the fact that in the basic matrix multiplication, the access to the data is non aligned and sparse. Moreover, this algorithm isn't efficient because we access several times to the same data in the different matrices A and B.

* The tiled multiplication matrix use the shared memory capability of GPU in order to solve this problem. Thanks to the shared memory, we only access once the data by synchronizing every threads within a block. In fact, the performances are much better than the basic multiplication matrix.

* Finally we compared the average computation time with the built-in function cublasSgemm from cuBLAS. This library is over optimized and perform the best computation time. This function should implement a tiled algorithm, but it certainly use more optimizations.

* I can't precisely explain why a smaller tile width gives a better performance. Maybe this is due to bank conflicts.